Link: https://www.udemy.com/course/langchain-in-action-develop-llm-powered-applications/

OpenAI´s function_Calling was first deprecated in favor of tool_calling - but now the correct terms seems to be function_calling again.
If you call it tool- or function calling - at the moment is does not really matter. 

Reference: https://platform.openai.com/docs/guides/function-calling

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
print(llm.invoke("How will the weather be in munich today?"))

content="I'm unable to provide real-time weather updates. To find out the current weather in Munich, I recommend checking a reliable weather website or app for the latest information." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 17, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_aa07c96156', 'finish_reason': 'stop', 'logprobs': None} id='run-f9a357d2-687c-4719-8507-3b51f4d28f13-0' usage_metadata={'input_tokens': 17, 'output_tokens': 32, 'total_tokens': 49, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [4]:
from langchain_core.tools import tool


@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a specified city.

    Args:
        city (str): The name of the city where you want to check the weather.

    Returns:
        str: A description of the current weather in the specified city.
    """
    return "Sunny, 22°C"


@tool
def outdoor_seating_availability(city: str) -> str:
    """
    Check if outdoor seating is available at a specified restaurant in a given city.

    Args:
        city (str): The name of the city where you want to check for outdoor seating availability.

    Returns:
        str: A message stating whether outdoor seating is available or not.
    """
    return "Outdoor seating is available."


tools = [fake_weather_api, outdoor_seating_availability]

In [21]:
outdoor_seating_availability.description

'Check if outdoor seating is available at a specified restaurant in a given city.\n\nArgs:\n    city (str): The name of the city where you want to check for outdoor seating availability.\n\nReturns:\n    str: A message stating whether outdoor seating is available or not.'

In [22]:
outdoor_seating_availability.invoke({'city': 'Munich'})

'Outdoor seating is available.'

In [23]:
fake_weather_api.invoke({'city': 'Munich'})

'Sunny, 22°C'

In [6]:
fake_weather_api.description

'Check the weather in a specified city.\n\nArgs:\n    city (str): The name of the city where you want to check the weather.\n\nReturns:\n    str: A description of the current weather in the specified city.'

In [5]:
# from langchain_core.pydantic_v1 import BaseModel, Field

# class WeatherCheck(BaseModel):
#     """Check the weather in a specified city."""

#     city: str = Field(..., description="Name of the city to check the weather for")


# class OutdoorSeatingCheck(BaseModel):
#     """Check if outdoor seating is available at a specified restaurant in a given city."""

#     city: str = Field(..., description="Name of the city where the restaurant is located")


# tools = [WeatherCheck, OutdoorSeatingCheck]

In [7]:
llm_with_tools = llm.bind_tools(tools)

In [8]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
print(result)

content='' additional_kwargs={'tool_calls': [{'id': 'call_Ke17IqewUQ3v69CiU4IDu6gH', 'function': {'arguments': '{"city":"Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 158, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_11f3029f6b', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-80b9b8ab-0488-49dc-ba83-5314ed09e279-0' tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_Ke17IqewUQ3v69CiU4IDu6gH', 'type': 'tool_call'}] usage_metadata={'input_tokens': 158, 'output_tokens': 16, 'total_tokens': 174, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}

In [10]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)
print(result)

content='' additional_kwargs={'tool_calls': [{'id': 'call_zcuzQXxm5QyaErGaQk4ypsJH', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_I8Obibx19Mze53AP2xF2ghqH', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 166, 'total_tokens': 217, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_11f3029f6b', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-8ddd3574-10dd-4b36-b26d-7a69c64354cb-0' tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_zcuzQXxm5QyaErGaQk4ypsJH', 'type': 'tool_call'}, {'name': 'outdoor_seating_availabi

In [12]:
result.model_dump()

{'content': '',
 'additional_kwargs': {'tool_calls': [{'id': 'call_zcuzQXxm5QyaErGaQk4ypsJH',
    'function': {'arguments': '{"city": "Munich"}',
     'name': 'fake_weather_api'},
    'type': 'function'},
   {'id': 'call_I8Obibx19Mze53AP2xF2ghqH',
    'function': {'arguments': '{"city": "Munich"}',
     'name': 'outdoor_seating_availability'},
    'type': 'function'}],
  'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 51,
   'prompt_tokens': 166,
   'total_tokens': 217,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_11f3029f6b',
  'finish_reason': 'tool_calls',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-8ddd3574-10dd-4b36-b26d-7a69c64354cb-0',
 'example': False,
 'tool_calls': [{'name': 'fake_w

In [13]:
result.tool_calls

[{'name': 'fake_weather_api',
  'args': {'city': 'Munich'},
  'id': 'call_zcuzQXxm5QyaErGaQk4ypsJH',
  'type': 'tool_call'},
 {'name': 'outdoor_seating_availability',
  'args': {'city': 'Munich'},
  'id': 'call_I8Obibx19Mze53AP2xF2ghqH',
  'type': 'tool_call'}]

In [14]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? I would like to eat outside if possible"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [15]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_GblmMx1PF3n97sC6QBOLQwIg', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_lAd6w1cRhEg36Op66d46Gt1K', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 166, 'total_tokens': 217, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_11f3029f6b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-47f44da0-c4fb-4e82-aaf6-846a716c6db5-0'

In [16]:
tool_mapping = {
    "fake_weather_api": fake_weather_api,
    "outdoor_seating_availability": outdoor_seating_availability,
}

In [24]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [25]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_GblmMx1PF3n97sC6QBOLQwIg', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_lAd6w1cRhEg36Op66d46Gt1K', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 166, 'total_tokens': 217, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_11f3029f6b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-47f44da0-c4fb-4e82-aaf6-846a716c6db5-0'

In [26]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Munich today is sunny with a temperature of 22°C. Additionally, outdoor seating is available, so you can enjoy your meal outside!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 245, 'total_tokens': 276, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_11f3029f6b', 'finish_reason': 'stop', 'logprobs': None}, id='run-e2e621e7-99e8-4f23-9c3a-245f654c7636-0', usage_metadata={'input_tokens': 245, 'output_tokens': 31, 'total_tokens': 276, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})